In [11]:
import pandas as pd
import numpy as np
import geopandas as gpd
import pyproj
import shapely
#can use the constructive operation make_valid to check and fix invalid geometries
import leafmap as leafmap
import dash_leaflet as dl #from tutorial, replace with leafmap later
#dcc = dash_core_components
#hmtl = dash_html_components
from dash import Dash, html, dcc, callback, Output, Input, dash_table
import dash_bootstrap_components as dbc
import plotly.express as px
import json


In [12]:
practice_csv = pd.read_csv('data/terrawise/terrawise-practice-info.csv', usecols=['label','Year', 'Land use', 'Urea', 'Lime','Synthetic_fertiliser','Biochar'])

#put this in a loop instead of repeating all this code x3
geom_gdf = gpd.read_file('data/terrawise/Stratification/1/cea part/cea_1.shp')
geom_gdf.to_crs("WGS84", inplace=True)
geom2_gdf = gpd.read_file('data/terrawise/Stratification/2/cea part/cea_2.shp')
geom2_gdf.to_crs("WGS84", inplace=True)
geom3_gdf = gpd.read_file('data/terrawise/Stratification/3/cea part/cea_3.shp')
geom3_gdf.to_crs("WGS84", inplace=True)


In [13]:
geodf=gpd.read_file('data/terrawise/Stratification/1/cea part/cea_1.shp')
geodf.to_crs("WGS84", inplace=True)
geodf['rand'] = np.random.randint(1, 100, len(geodf))
#print(geodf)
#print("CRS: ", geodf.crs)
#print(geodf.geometry.head())

# points = geodf.geometry.centroid
# geodf['lat'] = points.apply(lambda x : x.y if x else np.nan)
# geodf['lon'] = points.apply(lambda x : x.x if x else np.nan)

#geojson=geodf.__geo_interface__
#geojson['features'][0]

In [14]:


# fig = px.choropleth_mapbox(
#     data_frame=geodf.set_index('FID'),
#     geojson=geodf.geometry,
#     locations=geodf.index,
#     mapbox_style = 'open-street-map',
#     center=dict(lat=-33.67, lon=118.2),
#     zoom=11,
#     color='rand'

# )

# fig.show()

In [15]:


input_geom=[geom_gdf, geom2_gdf, geom3_gdf]
output_geom=[]

#loop to attach practice data to each CEA
for i,geom in enumerate (input_geom):
    #print("STARTING GEOM: \n",geom)
    cea=i+1  #ideally this is programmatic, but for now setting as 1 so index starts right
    frame = geom.dissolve()
    frame['label']=cea
    frame_pracice_data = practice_csv[practice_csv['label']==cea]
    geom_combined = frame.set_index('label').join(frame_pracice_data.set_index('label'))
    output_geom.append(geom_combined)

#print("DATA PRACTICE GEOM: \n",output_geom[2])

#geojson_output=[]
#for i,geom in enumerate(output_geom):
#    geom_geojson=json.loads(geom.to_json)
#    geojson_output.append(geom_geojson)



In [16]:
geodf = pd.concat([output_geom[0], output_geom[1], output_geom[2]])
# #print(df)
# fig = px.choropleth_mapbox(
#     data_frame=geodf.set_index('FID'),
#     geojson=geodf.geometry,
#     locations=geodf.index,
#     mapbox_style = 'open-street-map',
#     center=dict(lat=-33.69, lon=118.2),
#     zoom=10,
#     color='Urea'
# )

# fig.show()

In [17]:
#dash app version 2

app = Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])


controls = dbc.Card(
    [
        html.Div(
            [
                dbc.Label("Select management practice"),
                dcc.Dropdown(practice_csv.columns, id='candidate'),
            ]
        ),
    ],
    body=True,
)





In [18]:

# app.layout = html.Div([
#     html.H4('Practice management by CEA'),
#     html.P("Select a management practice:"),
#     dcc.Dropdown(practice_csv.columns, id='candidate'),
#     dcc.Graph(id="map"),
#     dcc.Graph(figure={}, id='graph'),
    
# ])

app.layout = dbc.Container(
    [
        html.H1("Practice management by CEA"),
        html.Hr(),
        dbc.Row([dbc.Col(dcc.Graph(id="map"), md=6),
                dbc.Col(dcc.Graph(figure={}, id='graph'), md=4)]),
        dbc.Row([dbc.Col(controls, md=10),],
            align="center",)
    ],
    fluid=True,
)


@app.callback(
    [Output("graph", "figure"),
     Output("map", "figure")], 
    Input('candidate', 'value'))


def display_choropleth(candidate):
    graph = px.histogram(practice_csv, x=candidate, y='label', histfunc='count')
    geodf = pd.concat([output_geom[0], output_geom[1], output_geom[2]])
    fig = px.choropleth_mapbox(
        data_frame=geodf.set_index('FID'),
        geojson=geodf.geometry,
        locations=geodf.index,
        mapbox_style = 'open-street-map',
        center=dict(lat=-33.69, lon=118.2),
        zoom=10,
        color=candidate
    )
    return graph,fig

if __name__ == '__main__':
    app.run_server(debug=False, port=8051)


In [19]:
# #build dash components
# app = Dash(__name__)

# app.layout = html.Div([
#     html.Div(children='Practice management data'),
#     dash_table.DataTable(data=practice_csv.to_dict('records'), page_size=3),
#     dcc.Dropdown(practice_csv.columns, id='controls-practice-dropdown'),
#     dcc.Graph(figure={}, id='graph-by-practice'),
#     html.Div([
#         html.Div(children='Mapped data (not currently linked)'),

#         dl.Map(children=[
#                 dl.TileLayer(),
#                 dl.GeoJSON(data=geom_geojson,
#                     zoomToBounds=True,
#                     id='shapes')],
#                 id='map',
#                 style={'height': '40vh'},
#                 center=[-31, 120], #replace with centroid and extent of geometry data
#                 zoom=4,
#                 zoomControl=True,
#                 doubleClickZoom=False,
#                 dragging=False,
#                 scrollWheelZoom=True,
#                 preferCanvas=True)],
#             id='map_div', className='map_div')])

# @callback(
#     Output(component_id='graph-by-practice', component_property='figure'),
#     Input(component_id='controls-practice-dropdown', component_property='value')
# )

# def update_graph(col_chosen):
#     fig = px.histogram(practice_csv, x=col_chosen, y='label', histfunc='count')
#     return fig

# if __name__ == '__main__':
#     app.run(debug=True, port=8051)
